In [9]:
import oracledb
import pandas as pd
import random
import string
import requests
from datetime import datetime, timedelta

In [10]:
# roles = {
#     "SUPER_ADMIN": "모든 기능 허용",
#     "USER_MANAGE": "회원 관리",
#     "PRODUCT_MANAGE": "상품 관리",
#     "ORDER_MANAGE": "주문/배송 관리",
#     "CONTENT_MANAGE": "게시글/공지/배너 관리"
# }

### 관리자

- 총 관리자 13명 (유효한 관리자 10명, 비활성화 관리자 3명)
- 가입 날짜 : 3달 전 입사 ~ 6년 전 입사
- 계정 상태 : 활성, 비활성(퇴사로 인해)
- 최근 로그인 시간 : 오늘(2025-12-07) ~ 5일 전까지

In [11]:
oracledb.init_oracle_client(lib_dir=r"D:\\instantclient_23_9")

conn = oracledb.connect(
    user="adsql",          # 사용자명
    password="oracle_4U",      # 비밀번호
    dsn="localhost:1521/xe" # 접속 정보 (SQL Developer와 동일)
)
cur = conn.cursor()

In [12]:
# 한국 이름 반환
def return_korean_name():
	name_df = pd.read_csv('한국인이름.csv')
	return random.choice(name_df['한국인이름'])

In [13]:
# 랜덤으로 이메일 생성
def random_email():
    domains = ["gmail.com", "naver.com", "daum.net", "hotmail.com", "yahoo.com"]
    name_length = random.randint(6, 12)  # 아이디 길이
    name = ''.join(random.choices(string.ascii_lowercase + string.digits, k=name_length))
    domain = random.choice(domains)
    return f"{name}@{domain}"

In [14]:
# 랜덤으로 비밀번호 생성
def random_pwd():
    len_pwd = random.randint(12,16)  # 비밀번호 길이
    pwd = ''.join(random.choices(string.ascii_lowercase + string.digits + random.choice(string.punctuation), k=len_pwd))
    return pwd

In [15]:
roles = ["SUPER_ADMIN", "USER_MANAGE", "PRODUCT_MANAGE",
         "ORDER_MANAGE","CONTENT_MANAGE"]

def random_signin_date():
	return datetime.today() - timedelta(random.randint(100,2190))
  
def recently_login():
	return datetime.today() - timedelta(minutes=random.randint(10,4320))

In [16]:
manager_data = []

for i in range(1,51):
    data = [
		i,
		return_korean_name(),
		random_email(),
		random_pwd(),
		random.choice(roles),
		random_signin_date(),
		random.choices(['활성','비활성'],weights=[0.9,0.1])[0],
		recently_login()
	]
    
    manager_data.append(data)

In [17]:
m = pd.read_sql('select * from 관리자', conn)
m.columns

C:\Users\amysm\AppData\Local\Temp\ipykernel_12104\1841619707.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  m = pd.read_sql('select * from 관리자', conn)


Index(['관리자ID', '관리자이름', '이메일', '비밀번호', '권한', '계정생성일', '계정상태', '최근로그인시간'], dtype='object')

In [18]:
manager = pd.DataFrame(manager_data, columns=m.columns)
manager.to_csv('데이터_관리자테이블.csv',index=False)

In [19]:
manager['권한'].unique()

array(['ORDER_MANAGE', 'SUPER_ADMIN', 'USER_MANAGE', 'CONTENT_MANAGE',
       'PRODUCT_MANAGE'], dtype=object)